In [1]:

%matplotlib widget
import matplotlib
# List all paths in data folder
import os
data_dir = "C:/Users/gator/OneDrive - University of Florida/10x images for quantification/Manual Counts"
file_names = []
for file_name in os.listdir(data_dir):
    file_names.append(file_name)
    print(file_name)

1sAc2r1 PM NEUN AF.roi
1sAc2r1 PM NEUN DW.roi
1sAc2r1 PM NEUN.tif
1sAc4r3 PM NEUN DW.roi
1sAc4r3 PM NEUN.tif
2sAc4r5 PM NEUN AF.roi
2sAc4r5 PM NEUN DW.roi
2sAc4r5 PM NEUN.tif
2sBc4r1 PM NEUN AF.roi
2sBc4r1 PM NEUN DW.roi
2sBc4r1 PM NEUN.tif
2sBc5r3 PM NEUN AF.roi
2sBc5r3 PM NEUN DW.roi
2sBc5r3 PM NEUN.tif
2sCc1r1 PM NEUN AF.roi
2sCc1r1 PM NEUN DW.roi
2sCc1r1 PM NEUN.tif
3sAc2r3 PM NEUN AF.roi
3sAc2r3 PM NEUN DW.roi
3sAc2r3 PM NEUN.tif
3sAc3r2 PM NEUN AF.roi
3sAc3r2 PM NEUN DW.roi
3sAc3r2 PM NEUN.tif
3sCc6r2 PM NEUN AF.roi
3sCc6r2 PM NEUN DW.roi
3sCc6r2 PM NEUN.tif
4sBc1r2 PM NEUN AF.roi
4sBc1r2 PM NEUN DW.roi
4sBc1r2 PM NEUN.tif
cropped


In [2]:

from roifile import ImagejRoi
annotation_dict = {}
for file_name in file_names:
    if(".roi" in file_name):
        file_name_n_ext = file_name.split(".")[0]
        file_name_split = file_name_n_ext.split(" ")
        annotator = file_name_split[-1]
        stain = file_name_split[-2]
        image_id = file_name_split[0]
        image_name = image_id + " " + file_name_split[1] + " " + stain
        roi = ImagejRoi.fromfile(data_dir + "/" + file_name)
        if image_name not in annotation_dict.keys():
            annotation_dict[image_name] = {}
            annotation_dict[image_name]["annotators"] = []
            annotation_dict[image_name]["rois"] = []
        annotation_dict[image_name]["annotators"].append(annotator)
        annotation_dict[image_name]["rois"].append(roi)
print(annotation_dict)

{'1sAc2r1 PM NEUN': {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '1sAc2r1 PM NEUN AF'>, <ImagejRoi '1sAc2r1 PM NEUN DW'>]}, '1sAc4r3 PM NEUN': {'annotators': ['DW'], 'rois': [<ImagejRoi '1sAc4r3 PM NEUN DW'>]}, '2sAc4r5 PM NEUN': {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '2sAc4r5 PM NEUN AF'>, <ImagejRoi '2sAc4r5 PM NEUN DW'>]}, '2sBc4r1 PM NEUN': {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '2sBc4r1 PM NEUN AF'>, <ImagejRoi '2sBc4r1 PM NEUN DW'>]}, '2sBc5r3 PM NEUN': {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '2sBc5r3 PM NEUN AF'>, <ImagejRoi '2sBc5r3 PM NEUN DW'>]}, '2sCc1r1 PM NEUN': {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '2sCc1r1 PM NEUN AF'>, <ImagejRoi '2sCc1r1 PM NEUN DW'>]}, '3sAc2r3 PM NEUN': {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '3sAc2r3 PM NEUN AF'>, <ImagejRoi '3sAc2r3 PM NEUN DW'>]}, '3sAc3r2 PM NEUN': {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '3sAc3r2 PM NEUN AF'>, <ImagejRoi '3sAc3r2 PM NEUN DW'>]}, '3sCc6r2 PM NEUN': {'an

In [3]:
import math
from matplotlib import pyplot as plt
from skimage import io
import copy

min_required_distance = 20
min_agreeance = 0.5
image_coord_pairs = {}
for idx, (image_name, annotators) in enumerate(annotation_dict.items()):
    image_coord_pairs[image_name] = {
        "annotators" : [],
        "unmatched" : [],
        "matched" : [],
    }
    compiled_coords = []
    for annotator, roi in list(zip(annotators["annotators"],annotators["rois"])):
        image_coord_pairs[image_name]["annotators"].append(annotator)
        compiled_coords.append(roi.coordinates().tolist())
    print(image_name, annotators)
    coord_matches = []
    temp_coords = copy.deepcopy(compiled_coords)
    annotator_index = 0
    coord_index = 0
    for i in range(len(temp_coords)):
        coord_matches.append([])
    while(coord_index<len(temp_coords[annotator_index])):
        current_cord = temp_coords[annotator_index][coord_index]
        match = [current_cord]
        match_indexes = [coord_index]
        match_count = 1
        for index_offset in range(len(temp_coords)-1):
            other_annotator_index = annotator_index+index_offset+1
            if(other_annotator_index>len(temp_coords)-1):
                other_annotator_index -= len(temp_coords)
            
            min_distance = 100000000
            min_coord = []
            min_coord_idx = 0
            for idx, other_coord in enumerate(temp_coords[other_annotator_index]):
                distance_to_coord = math.sqrt(math.pow(other_coord[0]-current_cord[0],2) + math.pow(other_coord[1]-current_cord[1],2))
                if(distance_to_coord<min_distance):
                    min_distance = distance_to_coord
                    min_coord = other_coord
                    min_coord_idx = idx
            if(min_distance<min_required_distance):
                match_count+=1
                match.append(min_coord)
                match_indexes.append(min_coord_idx)
            else:
                match.append([])
                match_indexes.append(None)

        if(match_count/len(temp_coords)>min_agreeance):
            coord_matches[annotator_index].append(match)
            for index_offset in range(len(temp_coords)):
                other_annotator_index = annotator_index+index_offset
                if(other_annotator_index>len(temp_coords)-1):
                    other_annotator_index -= len(temp_coords)
                temp_coords[other_annotator_index].pop(match_indexes[index_offset])
        else:
            coord_index+=1
    for annotator_coords in temp_coords:
        image_coord_pairs[image_name]["unmatched"].append(annotator_coords)
    image_coord_pairs[image_name]["matched"] = coord_matches
    print("Matches")
    for coord in coord_matches:
        print("Found num of matches: " + str(len(coord)))

1sAc2r1 PM NEUN {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '1sAc2r1 PM NEUN AF'>, <ImagejRoi '1sAc2r1 PM NEUN DW'>]}
Matches
Found num of matches: 233
Found num of matches: 0
1sAc4r3 PM NEUN {'annotators': ['DW'], 'rois': [<ImagejRoi '1sAc4r3 PM NEUN DW'>]}
Matches
Found num of matches: 501
2sAc4r5 PM NEUN {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '2sAc4r5 PM NEUN AF'>, <ImagejRoi '2sAc4r5 PM NEUN DW'>]}
Matches
Found num of matches: 499
Found num of matches: 0
2sBc4r1 PM NEUN {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '2sBc4r1 PM NEUN AF'>, <ImagejRoi '2sBc4r1 PM NEUN DW'>]}
Matches
Found num of matches: 123
Found num of matches: 0
2sBc5r3 PM NEUN {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '2sBc5r3 PM NEUN AF'>, <ImagejRoi '2sBc5r3 PM NEUN DW'>]}
Matches
Found num of matches: 57
Found num of matches: 0
2sCc1r1 PM NEUN {'annotators': ['AF', 'DW'], 'rois': [<ImagejRoi '2sCc1r1 PM NEUN AF'>, <ImagejRoi '2sCc1r1 PM NEUN DW'>]}
Matches
Found num of matches: 225
Fou

In [6]:
# Save all annotationes to plot
# image_to_display = "all"
image_to_display = "none"
for image_name, annotation_dict in image_coord_pairs.items():
    print(image_name)
    if(image_to_display in image_name or image_to_display in "all"):
        image = io.imread(data_dir + "/" + image_name + ".tif")
        plt.cla()
        plt.imshow(image)
        colors = ["red", "blue", "yellow"]
        for idx, unmatched_coords in enumerate(image_coord_pairs[image_name]["unmatched"]):
            for coord in unmatched_coords:
                circle = plt.Circle(coord,10 ,alpha=0.2, fill = True , color=colors[idx], linewidth=0)
                plt.gca().add_artist(circle)
        for idx, matched_coords in enumerate(image_coord_pairs[image_name]["matched"]):
            for coord in matched_coords:
                circle = plt.Circle(coord[0],10 ,alpha=0.2, fill = True , color="green", linewidth=0)
                plt.gca().add_artist(circle)
        plt.savefig(image_name + " annotation plot.png", dpi=800)
        # plt.show()

1sAc2r1 PM NEUN
1sAc4r3 PM NEUN
2sAc4r5 PM NEUN
2sBc4r1 PM NEUN
2sBc5r3 PM NEUN
2sCc1r1 PM NEUN
3sAc2r3 PM NEUN
3sAc3r2 PM NEUN
3sCc6r2 PM NEUN
4sBc1r2 PM NEUN


In [5]:
# Convert and save images as heatmap
import os
from skimage import io
from scipy import stats
from roifile import ImagejRoi
from matplotlib import pyplot as plt
import numpy as np
from skimage.transform import resize
import cv2

dataset_dir = "C:/Users/gator/FullerLab/BrainStemSegmenter/Data_10-28-2022"
crop_image_shape = (300, 300)
image_scale = 1
patch_size = 32
noutputs = 1

def remove_coord(target_coord, coord_list):
    for idx, coord in enumerate(coord_list):
        if(target_coord[0]==coord[0]):
            if(target_coord[1]==coord[1]):
                coord_list = np.delete(coord_list, idx, 0)
    return coord_list

for image_name, annotation_dict in image_coord_pairs.items():
    image_path = data_dir + "/" + image_name + ".tif"
    cropped_path = data_dir + "/cropped"
    all_rois = []
    for idx, matched_coords in enumerate(annotation_dict["matched"]):
        for coord_pair in matched_coords:
            all_rois.append(coord_pair[0])
    
    cropped_path = data_dir + "/cropped"
    os.makedirs(cropped_path, exist_ok=True)
    image = io.imread(image_path)
    image_shape = image.shape
    
    rois_remaining = copy.deepcopy(all_rois)
    index = 0
    print("cropping and saving : " + image_name)
    while len(rois_remaining)>0:
        # print(len(rois_remaining))
        x, y = rois_remaining[0]
        left = int(x-(crop_image_shape[0]/2))
        top = int(y-(crop_image_shape[1]/2))
        right = int(x+(crop_image_shape[0]/2))
        bottom = int(y+(crop_image_shape[1]/2))
        crop_image = image[top:bottom,left:right]
        # print(crop_image.shape)
        rois_in_crop = []
        for roi in all_rois:
            roi_x, roi_y = roi
            if(top<roi_y<bottom and left<roi_x<right):
                rois_in_crop.append([roi_x-left, roi_y-top])
                rois_remaining = remove_coord(roi, rois_remaining)
        rois_in_crop = np.array(rois_in_crop)

        label_img = np.zeros((crop_image.shape[0]+(patch_size*image_scale), crop_image.shape[1]+(patch_size*image_scale))).astype(np.uint8)
        for x, y in rois_in_crop:
            left = int(x)
            top = int(y)
            right = int(x+(patch_size*image_scale))
            bottom = int(y+(patch_size*image_scale))
            label_img[top:bottom,left:right] += 1
        
        crop_image = cv2.resize(crop_image, dsize=(int(crop_image.shape[0]/image_scale), int(crop_image.shape[1]/image_scale)), interpolation=cv2.INTER_CUBIC)
        label_img = cv2.resize(label_img, dsize=(int(label_img.shape[0]/image_scale), int(label_img.shape[1]/image_scale)), interpolation=cv2.INTER_CUBIC)

        split_image_name = image_name.split(".")
        cropped_file_name = split_image_name[0] + "_cropped_count_" + str(len(rois_in_crop)) + "_" + str(index) + ".png"
        cropped_label_file_name = split_image_name[0] + "_cropped_label_count_" + str(len(rois_in_crop)) + "_" + str(index) + ".png"
        cropped_file_path = cropped_path + "/" + cropped_file_name
        cropped_label_file_path = cropped_path + "/" + cropped_label_file_name
        io.imsave(cropped_file_path, crop_image, check_contrast=False)
        io.imsave(cropped_label_file_path, label_img, check_contrast=False)
        print("saved: " + cropped_label_file_name)
        print("saved: " + cropped_file_name)
        index+=1



cropping and saving : 1sAc2r1 PM NEUN
saved: 1sAc2r1 PM NEUN_cropped_label_count_3_0.png
saved: 1sAc2r1 PM NEUN_cropped_count_3_0.png
saved: 1sAc2r1 PM NEUN_cropped_label_count_4_1.png
saved: 1sAc2r1 PM NEUN_cropped_count_4_1.png
saved: 1sAc2r1 PM NEUN_cropped_label_count_28_2.png
saved: 1sAc2r1 PM NEUN_cropped_count_28_2.png
saved: 1sAc2r1 PM NEUN_cropped_label_count_16_3.png
saved: 1sAc2r1 PM NEUN_cropped_count_16_3.png
saved: 1sAc2r1 PM NEUN_cropped_label_count_25_4.png
saved: 1sAc2r1 PM NEUN_cropped_count_25_4.png
saved: 1sAc2r1 PM NEUN_cropped_label_count_28_5.png
saved: 1sAc2r1 PM NEUN_cropped_count_28_5.png
saved: 1sAc2r1 PM NEUN_cropped_label_count_36_6.png
saved: 1sAc2r1 PM NEUN_cropped_count_36_6.png
saved: 1sAc2r1 PM NEUN_cropped_label_count_27_7.png
saved: 1sAc2r1 PM NEUN_cropped_count_27_7.png
saved: 1sAc2r1 PM NEUN_cropped_label_count_23_8.png
saved: 1sAc2r1 PM NEUN_cropped_count_23_8.png
saved: 1sAc2r1 PM NEUN_cropped_label_count_33_9.png
saved: 1sAc2r1 PM NEUN_cropped_c